# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [ ]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [1]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [2]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [3]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [4]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Encountered error: asyncio.run() cannot be called from a running event loop


/usr/local/lib/python3.11/site-packages/llama_index/core/tools/calling.py:22: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  return ToolOutput(
/usr/local/lib/python3.11/site-packages/llama_index/core/tools/calling.py:22: RuntimeWarning: coroutine 'Dispatcher.span.<locals>.async_wrapper' was never awaited
  return ToolOutput(


=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "agent roles in MetaGPT"}
=== Function Output ===
In MetaGPT, the agent roles include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and expertise tailored to different aspects of problem-solving and software development within the framework. The agents are specialized to perform distinct tasks efficiently, contributing to the overall effectiveness of the multi-agent system.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Encountered error: asyncio.run() cannot be called from a running event loop
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other by utilizing a shared messag

In [5]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 9
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-08-08
last_modified_date: 2024-06-24

Preprint
Table 2: Comparison of capabilities for MetaGPT and other approaches. ‘!’ indicates the
presence of a specific feature in the corresponding framework, ‘ %’ its absence.
Framework Capabiliy AutoGPT LangChain AgentVerse ChatDev MetaGPT
PRD generation % % % % !
Tenical design genenration % % % % !
API interface generation % % % % !
Code generation ! ! ! ! !
Precompilation execution % % % % !
Role-based task management % % % ! !
Code review % % ! ! !
Table 3: Ablation study on roles. ‘#’ denotes ‘The number of’, ‘Product’ denotes ‘Product man-
ager’, and ‘Project’ denotes ‘Project manager’. ‘ !’ indicates the addition of a specific role. ‘Revi-
sions’ refers to ‘Human Revision Cost’.
Engineer Product Architect Project #Agents #Lines Expense Revisions Executability
! % % % 1 83.0 $ 0.915 10 1.0
! ! % % 2 112.0 $ 1.059 6

In [6]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
Encountered error: asyncio.run() cannot be called from a running event loop
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include the SoftwareDev dataset, which consists of 70 diverse software development tasks. Within this dataset, 11 tasks are detailed, with the first seven tasks being utilized in the main experiments discussed in the provided context.
=== LLM Response ===
The evaluation datasets used in MetaGPT include the SoftwareDev dataset, which consists of 70 diverse software development tasks. Within this dataset, 11 tasks are detailed, with the first seven tasks being utilized in the main experiments discussed in the pr

In [7]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "results over the SoftwareDev dataset in MetaGPT"}
=== Function Output ===
Encountered error: asyncio.run() cannot be called from a running event loop
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over the SoftwareDev dataset in MetaGPT"}
=== Function Output ===
MetaGPT achieves an average score of 3.9 over the SoftwareDev dataset, surpassing ChatDev's score of 2.1. It simplifies the process of transforming abstract requirements into detailed class and function designs through a specialized division of labor and SOPs workflow. Additionally, MetaGPT's structured messaging and feedback mechanisms reduce loss of communication information and improve the execution of code.
=== LLM Response ===
MetaGPT achieves an average score of 3.9 over the SoftwareDev dataset, surpassing ChatD

## Lower-Level: Debuggability and Control

In [8]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [9]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [10]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Encountered error: asyncio.run() cannot be called from a running event loop


In [11]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 8d2114d6-dba3-4753-8530-b1b14bdcd8fb: 1
asyncio.run() cannot be called from a running event loop


In [12]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 8d2114d6-dba3-4753-8530-b1b14bdcd8fb: 1


TaskStep(task_id='8d2114d6-dba3-4753-8530-b1b14bdcd8fb', step_id='4b8fb78c-de37-4b45-a522-918b5f781182', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [13]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Encountered error: asyncio.run() cannot be called from a running event loop


In [14]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
I encountered an error while trying to retrieve the information about how agents share information in MetaGPT. Let me try another approach to provide you with the necessary details.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "how agents share information", "page_numbers": ["5"]}
=== Function Output ===
Agents share information through structured communication interfaces in MetaGPT.
False


In [16]:
response = agent.finalize_response(task.task_id)

ValueError: finalize_response can only be called on the last step output

In [17]:
print(str(response))

assistant: MetaGPT achieves an average score of 3.9 over the SoftwareDev dataset, surpassing ChatDev's score of 2.1. It simplifies the process of transforming abstract requirements into detailed class and function designs through a specialized division of labor and SOPs workflow. Additionally, MetaGPT's structured messaging and feedback mechanisms reduce loss of communication information and improve the execution of code.
